#####  挑战1：完成环境配置，使可以进行节点编码步骤。（60分）
#####  挑战2：代码漏洞检测是一个图级的二分类问题，它在对图节点编码的基础上，进一步得到图级的编码。
#####        实现一个Readout层，得到图级的编码，从而可以完成漏洞检测模型的训练和评估步骤。（30分）
#####  挑战3：在使用所有训练数据且不修改训练集的划分条件下，不限定修改方式，使模型的预测准确率达到60%以上。
#####       （超过55%可以酌情给分，达成70%以上可以直接给满分）

In [ ]:
# 环境安装
# 1.安装并激活python环境, 已经验证的版本是3.7.2
!python --version 
!conda create -m devign python==3.7.2 #此命令不要重复执行，成功后可注释掉
!conda activate devign    
# 2.根据当前机器的情况安装torch、torch-geometric、torch-sparse 和 torch-scatter
#   具体安装方法参考 https://pytorch.org/get-started/locally/
# 3.安装当前试验所依赖的python包
!pip instlall -r requirements.txt

In [ ]:
# 原始的代码数据集存放在 data/raw (Paths.raw)中
# 每一条数据包含：
# project名，commit_id，target（是否为漏洞代码），func（函数代码文本）
# 我们探索一下原始数据的构成
import pandas as pd
dataset = pd.read_json("data/raw/dataset.json")
dataset.head(5)

In [ ]:
# 并使用joern工具将代码数据转化为CPG图
# 生成过程比较慢而且需要安装环境，在此省略
# 感兴趣的同学可以了解一下joern： https://docs.joern.io/home
# 生成的代码图数据存放在data/cpg (Paths.cpg)中，其中每100条数据写入一个文件
# 代码图数据预览
cpg_data_0 = pd.read_json("data/cpg/0_cpg.json")
print(cpg_data_0.head())
print("cpg 0:")
for k in cpg_data_0.iloc[0].functions:
    print(k, end=", ")
print("\n")
#print(cpg_data_0.iloc[0].functions["AST"])
#TODO 画出这个CPG图

In [ ]:
# 训练用于编码节点中文本的word2vec模型，并对节点的文本进行编码
from main import main
main.embed_task()

In [ ]:
# 模型训练和验证
main.process_task(True, False)